### UHF vs RHF
UHF has less restrictions than RHF. The main difference is that the functions are no longer restricted to only doubly occupied functions. Now there is a possibility to have a different number of $\alpha$ and $\beta$ spins. This implies a reduction of the symmetry of the wave-function. The wave-function will no longer be an eigenfunction of the $S_{z}$ operator. The reason to use UHF is that it will result in a lower energy.

## Implementation
Major parts of the code can just be copied from the RHF implementation. The general framework of the problem is the same, there are just some changes in the restrictions.

In [2]:
# ==> Import Psi4 & NumPy <==
from pyscf import gto, scf
import numpy as np
import scipy

mol = gto.M(atom = 'O 0.0 0.0 0.0; H 1.0 0.0 0.0; H 0.0 1.0 0.0', basis = 'ccpvdz')
MAXITER = 40
E_conv = 1.0e-6

In [3]:
S = mol.intor('int1e_ovlp')
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H_core = T + V
eri = mol.intor('int2e')

In [4]:
enuc = mol.get_enuc()
ndocc = mol.nelec[0]

At this point it is necessary to implement some chenges. We no longer work with the number of doubly occupied orbtials because in UHF it is possible to have single occupied orbitals. There is a different Fock-operator for the alpha spins ($F_{\alpha}$) and the beta spins ($F_{\beta}$).

In [8]:
eps, C = scipy.linalg.eigh(H_core, S)
n_alpha = mol.nelec[0]
n_beta = mol.nelec[1]

Calpha = C[:, :n_alpha]
Cbeta = C[:, :n_beta]

D_alpha = np.einsum('pi,qi->pq', Calpha, Calpha, optimize=True)
D_beta = np.einsum('pi,qi->pq', Cbeta, Cbeta, optimize=True)

